# 1. Подготовка данных <a id='section1'></a>

Импортируем необходимые библиотеки

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re 
from nltk.stem import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

Загрузим данные

In [4]:
df = pd.read_csv('datasets/toxic_comments.csv')

df.info()
df.head()

FileNotFoundError: [Errno 2] File datasets/toxic_comments.csv does not exist: 'datasets/toxic_comments.csv'

Создадим отдельный столбец для подготовки признаков

In [ ]:
df.tail

In [21]:
df['stemmed'] = df['text'].values.astype('U')

Почистим и стемматизируем тексты

In [22]:
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())

df['stemmed'] = df['stemmed'].apply(
    lambda x: clear_text(x))

In [23]:
stemmer = SnowballStemmer("english")

def stemmed(text):
    words = text.split(' ')
    stem_text = []
    for word in words:
        stem = stemmer.stem(word)
        stem_text.append(stem)
        "".join(stem) 
    stem_text = " ".join(stem_text)    
    return stem_text

In [24]:
%%time
df['stemmed'] = df['stemmed'].apply(stemmed)

Wall time: 1min 53s


Проверим данные

In [25]:
df.head()

,text,toxic,stemmed
0,Explanation\nWhy the edits made under my usern...,0,explan whi the edit made under my usernam hard...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m realli not tri to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make ani real suggest on improv i...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero ani chanc you rememb what ...


Определим признаки и разделим их на обучающие и тестовые

In [26]:
features = df['stemmed']
target = df['toxic']
 
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.1, 
                                                                            random_state=1234, stratify=target)

Векторизируем признаки

In [27]:
stop_words = set(nltk_stopwords.words('english'))

count_tf_idf_stem = TfidfVectorizer(stop_words = stop_words)
features_train = count_tf_idf_stem.fit_transform(features_train)

features_test = count_tf_idf_stem.transform(features_test)

### Вывод:
Данные проверены и обработанны. Признаки подготовлены. 

# 2. Обучение разных моделей с различными гиперпараметрами <a id='section2'></a>

### 2.1 Логистическая регрессия

Обучим модель, подготовим предсказания и вычислим F1-метрику

In [28]:
%%time
model_Linear = LogisticRegression(penalty='l1')
model_Linear.fit(features_train,target_train)
predictions = model_Linear.predict(features_test)
f1_score(target_test, predictions)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 1.43 s


0.7808077321366932

F1 = 0.78, что соответствует требованиям.

### 2.2 Случайный лес

Подберем оптимальные параметры

In [29]:
parameters_grid = {
    'n_estimators' : range(10,100,10),
    'min_samples_split' : range(2,10,1),
    'min_samples_leaf' : range(2,10,1),
    'max_depth' : range(2,30,2),
}

In [30]:
# %%time

# model_RFC = RandomForestClassifier()

# randomized_grid_cv = RandomizedSearchCV(model_RFC, parameters_grid, 
#                                                     scoring=make_scorer(f1_score,greater_is_better=True),
#                                                     n_iter=24                                                    
#                                                    )
# randomized_grid_cv.fit(features_train, target_train)
# randomized_grid_cv.best_params_

Wall time: 28min 54s

{'n_estimators': 40,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_depth': 26}

In [31]:
%%time
model_RFC = RandomForestClassifier(n_estimators=40, min_samples_split=5, min_samples_leaf=2, max_depth = 26)
model_RFC.fit(features_train,target_train)
predictions = model_RFC.predict(features_test)
f1_score(target_test, predictions)

Wall time: 4.02 s


0.011029411764705881

Судя по всему, подобранные параметры вызвали переобучение модели. Попробуем параметры по умолчанию.

In [32]:
%%time
model_RFC = RandomForestClassifier()
model_RFC.fit(features_train,target_train)
predictions = model_RFC.predict(features_test)
f1_score(target_test, predictions)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 1min 17s


0.6656188605108054

F1 = 0.67, что НЕ соответствует требованиям.

### 2.3 LightGBM

In [33]:
parameters_grid = {
    'max_depth' : range(5, 20, 1),
    'num_leaves' : range(70,130,5),
    'learning_rate' : np.arange(0.3, 0.9, 0.05),
    'n_estimators' : range(80, 120, 5),
}

In [34]:
# %%time

# model_LGBM = lgb.LGBMClassifier(n_jobs = -1)

# randomized_grid_cv = RandomizedSearchCV(model_LGBM, parameters_grid, n_iter=100,
#                                                     scoring=make_scorer(f1_score,greater_is_better=True))

# randomized_grid_cv.fit(features_train, target_train)
# randomized_grid_cv.best_params_

Wall time: 51min 11s

{'num_leaves': 75,
 'n_estimators': 105,
 'learning_rate': 0.4,
 'max_depth': 19}

In [35]:
%%time
model_LGBM = lgb.LGBMClassifier(n_jobs = -1, num_leaves=75, n_estimators=105, learning_rate=0.4, max_depth=19)
model_LGBM.fit(features_train,target_train)
predictions = model_LGBM.predict(features_test)
f1_score(target_test, predictions)

Wall time: 1min


0.76346284935242

F1 = 0.76. Результат соответствует требованиям. Но на всякий случай проверим модель с дефолтными параметрами. 

In [36]:
%%time
model_LGBM = lgb.LGBMClassifier()
model_LGBM.fit(features_train,target_train)
predictions = model_LGBM.predict(features_test)
f1_score(target_test, predictions)

Wall time: 58.5 s


0.7513340448239061

Качество предсказаний с подобранными параметрами немного лучше. 

#### Вывод:
Были подобраны оптимальные параметры для моделей. Лучшее качество предсказаний показала логистическая регрессия.

#### <a href='#istruction'>Перейти в начало</a>  

# 3. Общий вывод <a id='section3'></a>

Тексты были очищенны и стемматизированны. Признаки векторизированны. 

Для обучения были выбраны три модели: логистическая регрессия, случайный лес и LGBMClassifier. 

Для случайного леса и LGBMClassifier были подобраны гиперпараметры. Но в случае со случайным лесом это только ухудшило качество предсказаний. 

Лучшую метрику F1 показала логистическая регрессия (0.78). Она также обучалась и предсказывала в десятки раз быстрее других моделей. 

LGBMClassifier показал F1 = 0.76. 

Качество предсказаний случайного леса неудовлетворительное (0.66). Также эта модель оказалась самой медленной и нестабильной при подборе гиперпараметров. Судя по всему случайны лес не подходит для данного типа задач. 

#### <a href='#istruction'>Перейти в начало</a>  